# Plotting Vietnam Development Map

<img src="figures/vietnam-map-with-states.jpg" alt="drawing" style="width:200px;"/>

## Table of contents
1. [The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces](#1)<br>
    1.1 [Data](#11)<br>
    1.2 [Animated figures with Plotly Express](#12)<br>
2. [Proportion of poor households by region in Vietnam from 1998 to 2016 ](#2)<br>

5. [Important Notes](#333)<br>
5. [References](#666)<br>

In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

# Plot in browser (recommended)
import plotly.io as pio
pio.renderers.default = 'browser'

# Make Plotly work in your Jupyter Notebook
# import cufflinks as cf
# import chart_studio.plotly as py
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
# # Use Plotly locally
# cf.go_offline()

## 1. The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces <a id="1"></a>
Published by: Open Development Vietnam The data provides information on Vietnam's aquaculture production from 2013 to 2018. The aquaculture in Vietnam includes: farmed fish production, farmed shrimp production and other aquatic products. Aquaculture production is divided by province and city.

In [18]:
# import the Vietnam map by provinces data
vietnam_geo = json.load(open("geodata/aquaculture_production_of_vietnam.geojson", "r"))

# Convert map properties/
state_id_map = {}
for feature in vietnam_geo["features"]:
    feature["id"] = feature["properties"]["Code"]
    state_id_map[feature["properties"]["Name"]] = feature["id"]

# Import aquaculture_production csv
df = pd.read_csv("geodata/tsnt_2013__2018_en.csv")
df.rename(columns={"Provincial code":'Code',"Provincial name":'Name'},inplace=True)
years = ['2013','2014','2015','2016','2017','2018']
for i, y in enumerate(years):
    scale = 'ProdScale'+ y
    prod = 'Production'+y
    df[scale] = np.log10(df[y])
    df[prod] = df[y]

# Convert wide to long format
tsnt = df.drop(years, axis=1)
final_tsnt = pd.wide_to_long(tsnt, stubnames=['ProdScale','Production'], i=['Name','Code'], j="year")
final_tsnt.reset_index(inplace=True)

### 1.2 Choropleth map using GeoJSON

In [13]:
fig = px.choropleth(
    df,
    locations ="Code",
    geojson = vietnam_geo,
    color = "ProdScale2013",
    hover_name = "Name",
    hover_data = ["2013"],
    title ="The Aquaculture Production of Vietnam  2013 by Provinces"
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show()

<img src="figures/Aqua_prod_VN_2013.png" alt="drawing" style="width:900px;"/>

### 1.3 Animated figures with GeoJSON, Plotly Express <a id="12"></a>

In [19]:
fig = px.choropleth(
    final_tsnt,
    locations ="Code",
    animation_frame = "year",
    geojson = vietnam_geo,
    color = "ProdScale",
    hover_name = "Name",
    hover_data = ["Production"],
    title ="The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces",
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show('browser')

## 2. Proportion of poor households by region in Vietnam from 1998 to 2016  <a id="2"></a>
The dataset includes le pourcentage of poor households by region in Vietnam from 1998 to 2016. The standard of poor households for this period based on the average income per person per month of households is updated according to the consumer price index as follows: In 2010, VND 400,000 for rural areas and VND 500,000 for urban areas; Similarly, in 2013 it was VND 570,000 and VND 710,000; in 2014, VND 605,000 dong and VND 750,000; in 2015, there were VND 615,000 and VND 760,000 dong; In 2016, VND 630,000 and VND 780,000 respectively.

In [33]:
# import the Vietnam map by region data
vietnamregion_geo = json.load(open("geodata/poverty_rate_1998_2016.geojson", "r", encoding="utf8"))

# Convert map properties
region_id_map = {}
for feature in vietnamregion_geo["features"]:
    feature["row"] = feature["properties"]["row"]
    region_id_map[feature["properties"]["Name_EN"]] = feature["row"]


# Import aquaculture_production csv
poverty_rate_1998_2016 = pd.read_csv("geodata/poverty_rate_1998_2016.csv",delimiter="\t")
cols = sorted(poverty_rate_1998_2016.columns[3:], reverse=False)
# poverty_rate_1998_2016.drop(['row'], axis=1, inplace = True)

for i, y in enumerate(cols):
    scale = 'PovertyScale'+ y
    poverty = "Poverty" + y
    poverty_rate_1998_2016[scale] = poverty_rate_1998_2016[y]/100
    poverty_rate_1998_2016[poverty] = poverty_rate_1998_2016[y]

# Convert wide to long format
poverty = poverty_rate_1998_2016.drop(cols, axis=1)
final_poverty = pd.wide_to_long(poverty,['PovertyScale',"Poverty"], i=['Name_EN','Name_VI'], j="year")
final_poverty.reset_index(inplace=True)

In [37]:
poverty_rate_1998_2016

,Name_EN,Name_VI,row,2002,2004,2006,2008,2010,2011,2012,...,PovertyScale2012,Poverty2012,PovertyScale2013,Poverty2013,PovertyScale2014,Poverty2014,PovertyScale2015,Poverty2015,PovertyScale2016,Poverty2016
0,High Land,Tây Nguyên,1,52.4,51.8,29.2,24.0,21.0,22.2,20.3,...,0.203,20.3,0.178,17.8,0.162,16.2,0.138,13.8,0.113,11.3
1,Delta of Red River,Đồng bằng sông Hồng,2,30.7,21.5,12.7,10.0,8.6,8.3,7.1,...,0.071,7.1,0.060,6.0,0.049,4.9,0.040,4.0,0.032,3.2
2,Northern Midlands and Mountains,Trung du và Miền núi phía Bắc,3,64.5,47.9,29.4,27.5,25.1,29.4,26.7,...,0.267,26.7,0.238,23.8,0.219,21.9,0.184,18.4,0.160,16.0
3,North Central and Central Coast,Bắc Trung Bộ và Duyên hải miền Trung,4,42.5,35.7,25.3,22.2,19.2,20.4,18.5,...,0.185,18.5,0.161,16.1,0.140,14.0,0.118,11.8,0.098,9.8
4,South East,Đông Nam Bộ,5,7.6,8.2,4.6,3.1,2.5,2.3,1.7,...,0.017,1.7,0.013,1.3,0.011,1.1,0.010,1.0,0.007,0.7
5,Delta of Mekong River,Đồng bằng sông Cửu Long,6,36.9,23.4,15.3,13.0,11.4,12.6,11.6,...,0.116,11.6,0.101,10.1,0.092,9.2,0.079,7.9,0.065,6.5


### 1.2 Choropleth map using GeoJSON <a id="21"></a>

In [35]:
fig = px.choropleth(
    final_poverty,
    locations = "row",
    geojson = vietnamregion_geo,
    color ="Poverty2016",
    hover_name = "Name_EN",
    hover_data = ['Poverty2016'],
    title ="Proportion of poor households by region in Vietnam 2016",
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show('browser')

ValueError: Value of 'hover_data_0' is not the name of a column in 'data_frame'. Expected one of ['Name_EN', 'Name_VI', 'year', 'row', 'PovertyScale', 'Poverty'] but received: Poverty2016

### 2.3 Animated figures with GeoJSON, Plotly Express <a id="22"></a>

In [22]:
fig = px.choropleth(
    final_poverty,
    locations ="Name_EN",
    animation_frame = "year",
    geojson = vietnamregion_geo,
    color ="PovertyScale",
    hover_name = "Name_EN",
    hover_data = ['Poverty'],
    title ="Proportion of poor households by region in Vietnam from 1998 to 2016",
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show('browser')

## Important Notes <a id="333"></a>
Defining redraw: Setting redraw: false is an optimization for scatter plots so that animate just makes changes without redrawing the whole plot. For other plot types, such as contour plots, every frame must be a total plot redraw, i.e. redraw: true.

## Reference <a id="666"></a>
For additional information and attributes for creating bubble charts in Plotly see: https://plotly.com/python/bubble-charts/. 
For more documentation on creating animations with Plotly, see https://plotly.com/python/#animations.
https://plotly.com/python/animations/